In [1]:
import sys
sys.path.append('../../../Common/')

import CommonYFinance, CommonBinance, CommonMT5, CommonMongoDBConnector, CommonBacktest

In [2]:
def detectSignal(symbol, from_date, to_date, timeframe):

    import pandas as pd
    import plotly.graph_objects as go
    import redis
    import numpy as np
    from datetime import datetime

    # ##############################################Step 1: Lấy dữ liệu##############################################
    data = CommonBinance.CommonBinance.loaddataBinance_FromTo(symbol, from_date, to_date, timeframe)
    
    # ##############################################Step 2: Lấy dữ liệu##############################################  
    # Thiết lập cửa sổ thời gian cho SMA và độ lệch chuẩn
    window = 20
    # Tính toán SMA cho giá đóng cửa
    data['SMA'] = data['Close'].rolling(window=window).mean()
    # Tính toán MACD
    # EMA ngắn hạn
    data['short_ema'] = data['Close'].ewm(span=12, adjust=False).mean()
    # EMA dài hạn
    data['long_ema']  = data['Close'].ewm(span=26, adjust=False).mean()
    # Tính MACD
    data['MACD'] = data['short_ema']  - data['long_ema']
    # Tính Signal Line
    data['Signal_Line'] = data['MACD'].ewm(span=9, adjust=False).mean()
    # Tạo cột tín hiệu mua/bán
    data['Buy_Signal'] = (data['MACD'] > data['Signal_Line']) & (data['Close'] > data['SMA'])
    data['Sell_Signal'] = (data['MACD'] < data['Signal_Line']) & (data['Close'] < data['SMA'])    

    # ##############################################Step 3: Đẩy dữ liệu qua Redis##############################################
    # Nếu có tín hiệu thì đẩy qua Redis
    # Datetime  Open    High    Low	Close   Volume  SMA short_ema   long_ema    MACD    Signal_Line Buy_Signal  Sell_Signal
    # Tạo kết nối Redis
    r = redis.Redis(host='localhost', port=6379, db=0)
    # Tạo hash key
    hash_key = 'OG_CRTO_FT_MACD,MA'
    last_record = data.iloc[-1]
   
    # Chuyển đổi record cuối cùng thành từ điển và lưu vào Redis dưới dạng hash
    if(last_record['Buy_Signal'] == True or last_record['Sell_Signal'] == True):
        for field, value in last_record.to_dict().items():
            # Chuyển đổi giá trị uint64 và Timestamp thành chuỗi
            if isinstance(value, pd.Timestamp):
                value = value.isoformat()
            elif isinstance(value, (int, np.uint64)):
                value = str(value)
            r.hset(hash_key, field, value)  
            r.hset(hash_key, 'Symbol', symbol)
            r.hset(hash_key, 'Insertdate', datetime.now().isoformat())
        print(last_record)   
    else:
        print('Không có tín hiệu!')
    # ##############################################Step 4: Vẽ biểu đồ##############################################  
    # data.set_index('Datetime', inplace=True)
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ giá đóng cửa và SMA
    # fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Giá Đóng Cửa', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['SMA'], mode='lines', name='SMA', line=dict(color='orange')))
    # # Vẽ các điểm cho tín hiệu mua
    # buy_signals = data[data['Buy_Signal']]
    # fig.add_trace(go.Scatter(x=buy_signals.index, y=buy_signals['Close'], mode='markers', name='Buy Signal', marker=dict(symbol='triangle-up', size=10, color='green')))
    # # Vẽ các điểm cho tín hiệu bán
    # sell_signals = data[data['Sell_Signal']]
    # fig.add_trace(go.Scatter(x=sell_signals.index, y=sell_signals['Close'], mode='markers', name='Sell Signal', marker=dict(symbol='triangle-down', size=10, color='red')))
    # fig.update_layout(title='Giá Đóng Cửa và SMA', yaxis_title='Giá', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

    # ####################################################################################################
    # # Tạo figure
    # fig = go.Figure()
    # # Biểu đồ MACD và Signal Line
    # fig.add_trace(go.Scatter(x=data.index, y=data['MACD'], mode='lines', name='MACD', line=dict(color='blue')))
    # fig.add_trace(go.Scatter(x=data.index, y=data['Signal_Line'], mode='lines', name='Signal Line', line=dict(color='orange')))
    # fig.update_layout(title='MACD và Signal Line', yaxis_title='MACD', xaxis_rangeslider_visible=False, height=500)
    # # Hiển thị biểu đồ
    # fig.show()

In [3]:
# from datetime import datetime, timedelta
# import schedule
# import time

# def schedule_detectSignal():

#     symbol = 'EURUSD'
#     from_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
#     to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
#     timeframe = 15 # 1m
#     detectSignal(symbol, from_date, to_date, timeframe)

# # Lên lịch hàm scan_market để chạy mỗi 15 phút
# schedule.every(15).minutes.do(schedule_detectSignal)

# while True:
#     schedule.run_pending()  # Hàm này được gọi liên tục để kiểm tra xem có công việc nào đã đến thời gian cần thực hiện hay không
#     time.sleep(10)

In [4]:
from datetime import datetime, timedelta
import schedule
import time
import random

def schedule_detectSignal():
    symbol = 'ATAUSDT'

    from_date = (datetime.now() - timedelta(days=2)).strftime('%Y-%m-%d')  # Lấy ngày hôm qua
    to_date = (datetime.now() + timedelta(days=1)).strftime('%Y-%m-%d')
    timeframe = '1m' # 1m
    detectSignal(symbol, from_date, to_date, timeframe)

# Danh sách các phút cụ thể bạn muốn hàm được chạy
run_minutes = list(range(1, 60, 1))

while True:
    current_time = datetime.now()  
    current_minute = current_time.minute
    # Kiểm tra xem phút hiện tại có nằm trong danh sách các phút cần chạy hàm không
    if current_minute in run_minutes:
        # Kiểm tra xem đã chạy hàm trong phút hiện tại hay chưa
        last_run = getattr(schedule_detectSignal, 'last_run', None)
        if last_run is None or last_run != current_minute:
            schedule_detectSignal()
            # Lưu lại phút cuối cùng mà hàm đã chạy
            setattr(schedule_detectSignal, 'last_run', current_minute)   
                
    # Nghỉ 10 giây trước khi kiểm tra lại
    time.sleep(1)


Datetime       2024-01-13 10:00:00
Open                        0.0949
High                        0.0949
Low                         0.0944
Close                       0.0947
Volume                    275724.0
SMA                        0.09698
short_ema                 0.095708
long_ema                  0.096215
MACD                     -0.000506
Signal_Line              -0.000132
Buy_Signal                   False
Sell_Signal                   True
Name: 29, dtype: object
